# Detekcja Porzuconych Obiektów

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import glob

from AOD import DualBackgroundModel, BlobProcessor, AbandonmentDetector

## Konfiguracja

In [2]:
VIDEO_NUM = 5
VIDEO_PATH = f'Videos/Video{VIDEO_NUM}.avi'
OUTPUT_VIDEO_PATH = f'Outputs/AOD_Video{VIDEO_NUM}.avi'
os.makedirs('Outputs', exist_ok=True)

MIN_OBJECT_AREA = 50
MAX_OBJECT_AREA = 500

KNN_HISTORY = 20
KNN_DIST_THRESHOLD = 400.0
KNN_DETECT_SHADOWS = True

IOU_THRESHOLD = 0.3
IOD_FILTER_THRESHOLD = 0.1
STATIONARY_TIME_THRESHOLD = 300

## Główne przetwarzanie

In [3]:
cap = cv2.VideoCapture(VIDEO_PATH)
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video: {frame_width}x{frame_height} @ {fps} FPS")
print(f"Total frames: {total_frames} ({total_frames/fps:.1f} seconds)")

frames_base = f'Frames/Video{VIDEO_NUM}/'
os.makedirs(frames_base, exist_ok=True)

Video: 720x480 @ 29 FPS
Total frames: 3297 (113.7 seconds)


In [4]:
backSub = DualBackgroundModel(
    st_history=50,
    lt_history=5000, # Zwiększono z 1000, aby obiekt dłużej widniał na masce różnicy
    dist2Threshold=KNN_DIST_THRESHOLD,
    detectShadows=KNN_DETECT_SHADOWS,
    memory_length=200
)

blobProc = BlobProcessor(
    min_area=80,
    max_area=7000
)

detector = AbandonmentDetector()

In [5]:
frame_count = 0
paused = False

while cap.isOpened():
    if not paused:
        ret, frame = cap.read()
        if not ret: break
        
        frame_count += 1
        fgMask_st, fgMask_lt, dbMask = backSub.apply(frame, frame_count)
        
        pso_candidates = blobProc.process(dbMask, frame, frame_count)
        detector.update(pso_candidates, backSub, frame)

        display_frame = frame.copy()
        
        
        for obj_id, obj in detector.active_objects.items():
            x, y, w, h = obj.bbox
            
            cv2.rectangle(display_frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            if obj.max_contour is not None:
                try:
                    global_contour = obj.max_contour + [x, y]
                    cv2.drawContours(display_frame, [global_contour], -1, (0, 255, 0), 1)
                except Exception: pass

            label = f"ID:{obj_id} | Stab:{obj.stability_counter if hasattr(obj, 'stability_counter') else 'N/A'} | Status:{obj.type_name}"
            (text_w, text_h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            cv2.rectangle(display_frame, (x, y - 20), (x + text_w, y), (0, 0, 0), -1)
            cv2.putText(display_frame, label, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        
        
        cv2.imshow('Main Frame', display_frame)
        cv2.imshow('Difference Mask', dbMask) 
        
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): break
    elif key == ord('p'): paused = not paused

cap.release()
cv2.destroyAllWindows()

PSO ID 444 promoted to CSO ID 444
PSO ID 344 promoted to CSO ID 344
PSO ID 346 promoted to CSO ID 346
PSO ID 355 promoted to CSO ID 355
PSO ID 559 promoted to CSO ID 559
PSO ID 569 promoted to CSO ID 569
PSO ID 580 promoted to CSO ID 580
PSO ID 623 promoted to CSO ID 623
PSO ID 1002 promoted to CSO ID 1002
PSO ID 1683 promoted to CSO ID 1683
PSO ID 1903 promoted to CSO ID 1903
CSO ID 344 failed authentication (lost visual), removing.
CSO ID 346 failed authentication (lost visual), removing.
CSO ID 355 failed authentication (lost visual), removing.
CSO ID 559 failed authentication (lost visual), removing.
CSO ID 569 failed authentication (lost visual), removing.
CSO ID 580 failed authentication (lost visual), removing.
CSO ID 623 failed authentication (lost visual), removing.
PSO ID 3271 promoted to CSO ID 3271
PSO ID 3939 promoted to CSO ID 3939
PSO ID 5218 promoted to CSO ID 5218
PSO ID 5219 promoted to CSO ID 5219
PSO ID 5530 promoted to CSO ID 5530
PSO ID 5595 promoted to CSO ID 559

KeyboardInterrupt: 